In [ ]:
!rm -rf graph_text
!git clone https://github.com/mikhmakarov/graph_text.git

Cloning into 'graph_text'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 408 (delta 236), reused 292 (delta 127), pack-reused 0
Receiving objects: 100% (408/408), 6.23 MiB | 25.60 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [ ]:
!pip install sentence_transformers

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import sys

sys.path.append("graph_text")


from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# from models import TADW, TriDnr, DeepWalk, Node2Vec, Hope
from text_transformers import Ernie
from datasets import Cora, CiteseerM10, Dblp

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from collections import defaultdict

from task import Task

Going to train on cuda


In [ ]:
datasets = [
    ('Cora', Cora),
   # ('CiteseerM10', CiteseerM10),
  #   ('DBLP', Dblp)
]

test_ratios = [0.5, 0.7, 0.9, 0.95]


tasks = [
    ('Ernie', lambda ds: Task(ds, test_ratios, lambda: Ernie(), None, d=None, labels=False)),
    # ('BOW', lambda ds: Task(ds, test_ratios, lambda: BOW(), None, d=None, labels=False)),
    # ('TFIDF', lambda ds: Task(ds, test_ratios, lambda: TFIDF(), None, d=None, labels=False)),
    # ('LDA', lambda ds: Task(ds, test_ratios, lambda: LDA(), None, d=None, labels=False)),
    # ('SBERT pretrained', lambda ds: Task(ds, test_ratios, lambda: SBert(train=False, d=300), None, d=None, labels=False)),
    # ('W2V pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=False, d=300), None, d=None, labels=False)),
    # ('W2V (d=300)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=300), None, d=None, labels=False)),
    # ('W2V (d=64)', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), None, d=None, labels=False)),
    # ('Doc2Vec pretrained (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=False, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=300)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=300), None, d=None, labels=False)),
    # ('Doc2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Doc2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('Sent2Vec pretrained (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=False, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=600)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=600), None, d=None, labels=False)),
    # ('Sent2Vec (d=64)', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), None, d=None, labels=False)),
    # ('DeepWalk (d=100)', lambda ds: Task(ds, test_ratios, None, DeepWalk, d=100, labels=False)),
    # ('Node2Vec (d=100)', lambda ds: Task(ds, test_ratios, None, Node2Vec, d=100, labels=False)),
    # ('Hope (d=100)', lambda ds: Task(ds, test_ratios, None, Hope, d=100, labels=False)),
    # ('TADW - BOW', lambda ds: Task(ds, test_ratios, BOW, TADW, d=160, labels=False)),
    # ('TADW - TFIDF', lambda ds: Task(ds, test_ratios, TFIDF, TADW, d=160, labels=False)),
    # ('TADW - Sent2Vec', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), TADW, d=160, labels=False)),
    # ('TADW - Word2Vec', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), TADW, d=160, labels=False)),
    # ('TriDNR', lambda ds: Task(ds, test_ratios, None, TriDnr, d=160, labels=True)),
    # ('BOW:DeepWalk', lambda ds: Task(ds, test_ratios, BOW, DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Word2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: W2V(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
    # ('Sent2Vec:DeepWalk', lambda ds: Task(ds, test_ratios, lambda: Sent2Vec(train=True, d=64), DeepWalk, d=100,
    #                                       labels=False, concat=True)),
]

In [ ]:
res = {}

for ds_name, ds_constr in tqdm(datasets, desc='datasets'):
    ds = ds_constr()
    for task_name, task_constr in tqdm(tasks, desc='Tasks'):
        task = task_constr(ds)
        task_res = task.evaluate()
        for test_ratio in task_res:
            scores = task_res[test_ratio]
            res[f'{1 - test_ratio} - {ds_name} - {task_name}'] = scores

for name, scores in res.items():
    print(name, round(np.mean(scores), 3), round(np.std(scores), 3))



Tasks:   0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:00,  2.01it/s]

2it [00:01,  1.99it/s]

3it [00:01,  1.97it/s]

4it [00:01,  2.16it/s]

5it [00:02,  2.13it/s]

6it [00:02,  2.35it/s]

7it [00:03,  2.47it/s]

8it [00:03,  2.31it/s]

9it [00:04,  1.89it/s]

10it [00:04,  2.01it/s]

11it [00:05,  1.83it/s]

12it [00:05,  1.93it/s]

13it [00:06,  1.94it/s]

14it [00:06,  2.15it/s]

15it [00:07,  1.66it/s]

16it [00:08,  1.43it/s]

17it [00:09,  1.47it/s]

18it [00:09,  1.59it/s]

19it [00:10,  1.44it/s]

20it [00:11,  1.56it/s]

21it [00:11,  1.73it/s]

22it [00:12,  1.78it/s]

23it [00:12,  1.91it/s]

24it [00:13,  1.88it/s]

25it [00:13,  1.99it/s]

26it [00:14,  1.67it/s]

27it [00:14,  1.90it/s]

28it [00:15,  1.81it/s]

29it [00:15,  1.87it/s]

30it [00:16,  1.90it/s]

31it [00:16,  2.10it/s]

32it [00:16,  2.31it/s]

33it [00:17,  2.20it/s]

34it [00:17,  2.40it/s]

35it [00:18,  1.93it/s]

36it [00:18,  1.99it/s]

37it [00:19,  1.98it/s]

38it [00:19,  2.08

0.5 - Cora - Ernie [0.6432791728212703, 0.6654357459379616, 0.6521418020679468, 0.6617429837518464, 0.6491875923190547] 0.654357459379616 0.008144194240525978
0.30000000000000004 - Cora - Ernie [0.6054852320675106, 0.6223628691983122, 0.6118143459915611, 0.620253164556962, 0.620253164556962] 0.6160337552742614 0.006399051007638412
0.09999999999999998 - Cora - Ernie [0.5057424118129614, 0.5004101722723544, 0.5127153404429861, 0.5377358490566038, 0.5451189499589828] 0.5203445447087777 0.017804689661380733
0.050000000000000044 - Cora - Ernie [0.43179168286047415, 0.4232413525068014, 0.4224640497473766, 0.43878740769529734, 0.4741546832491255] 0.438087835211815 0.019002469231057605
